In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile

zip_path = '/content/drive/My Drive/Tree Species classification/archive.zip'  # Confirm this path
extract_to = '/content/tree_species_data'

with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_to)

print("✅ Dataset extracted successfully!")


✅ Dataset extracted successfully!


In [ ]:
!ls "/content/tree_species_data"


Tree_Species_Dataset


In [ ]:
train_path = '/content/tree_species_data/Tree Species Classification/train'
test_path = '/content/tree_species_data/Tree Species Classification/test'


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

base_dir = '/content/tree_species_data/Tree_Species_Dataset'
train_dir = '/content/tree_species_data/train'
test_dir = '/content/tree_species_data/test'

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_name)

    if os.path.isdir(class_path):
        images = [f for f in os.listdir(class_path) if f.endswith(('.jpeg', '.jpg', '.png'))]

        # Skip if class has fewer than 2 images
        if len(images) < 2:
            print(f"⚠️ Skipping class '{class_name}' due to insufficient images ({len(images)}).")
            continue

        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Create class folders in train and test
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        # Copy training images
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))

        # Copy test images
        for img in test_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, class_name, img))

print("✅ Dataset split complete!")


⚠️ Skipping class '.git' due to insufficient images (0).
✅ Dataset split complete!


In [ ]:
train_path = '/content/tree_species_data/train'
test_path = '/content/tree_species_data/test'


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 1280 images belonging to 30 classes.
Found 320 images belonging to 30 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

In [ ]:
# 🔹 Step 3: Setup Train/Test Paths
train_dir = os.path.join(extract_to, 'train')
test_dir = os.path.join(extract_to, 'test')

# 🔹 Step 4: Preprocess Images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)


Found 1280 images belonging to 30 classes.
Found 320 images belonging to 30 classes.


In [ ]:
# 🔹 Step 5: Create CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_data.num_classes, activation='softmax')
])

# 🔹 Step 6: Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,396,190 (28.21 MB)

 Trainable params: 7,396,190 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 📌 STEP 3: Build the CNN Model
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(train_set.num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 57600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │     7,372,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 30)             │         3,870 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,396,190 (28.21 MB)

 Trainable params: 7,396,190 (28.21 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 📌 STEP 4: Train the Model
history = model.fit(train_set,
                    epochs=10,
                    validation_data=test_set)


Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.0532 - loss: 3.8942 - val_accuracy: 0.0938 - val_loss: 3.2888
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 38s 950ms/step - accuracy: 0.0946 - loss: 3.2471 - val_accuracy: 0.1031 - val_loss: 3.1526
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.1268 - loss: 3.1209 - val_accuracy: 0.1781 - val_loss: 2.9559
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.1584 - loss: 2.9606 - val_accuracy: 0.2000 - val_loss: 2.9009
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 42s 1s/step - accuracy: 0.2223 - loss: 2.7629 - val_accuracy: 0.2000 - val_loss: 2.7334
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 39s 972ms/step - accuracy: 0.2304 - loss: 2.6850 - val_accuracy: 0.2406 - val_loss: 2.6695
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 40s 993ms/step - accuracy: 0.2777 - loss: 2.5551 - val_accuracy: 0.2719 - val_loss: 2.6365
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.3077 - loss: 2.4468 - val_accuracy: 0.2812 - 

In [ ]:
# 📌 STEP 5: Evaluate the Model
test_loss, test_acc = model.evaluate(test_set)
print("Test Accuracy:", test_acc)


10/10 ━━━━━━━━━━━━━━━━━━━━ 5s 497ms/step - accuracy: 0.3172 - loss: 2.5858
Test Accuracy: 0.30937498807907104


In [ ]:
# 📌 STEP 6: Save the Model (.h5 file)
model.save("tree_species_model.h5")


In [ ]:
from google.colab import files
files.download('model.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>